In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate(); Pkg.add("Flux")

using Flux.Tracker, Statistics, DelimitedFiles
using Flux.Tracker: Params, gradient, update!

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/notebooks/other/housing/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/notebooks/other/housing/Manifest.toml`
  [1520ce14] ↑ AbstractTrees v0.2.0 ⇒ v0.2.1
  [b99e7846] ↑ BinaryProvider v0.4.2 ⇒ v0.5.3
  [944b1d66] ↑ CodecZlib v0.5.0 ⇒ v0.5.1
  [3da002f7] ↑ ColorTypes v0.7.4 ⇒ v0.7.5
  [5ae59095] ↑ Colors v0.9.4 ⇒ v0.9.5
  [34da2185] ↑ Compat v1.1.0 ⇒ v1.4.0
  [864edb3b] ↑ DataStructures v0.12.0 ⇒ v0.14.0
  [f6369f11] ↑ ForwardDiff v0.9.0 ⇒ v0.10.1
  [e89f7d12] ↑ Media v0.4.1 ⇒ v0.5.0
  [e1d29d7a] ↑ Missings v0.3.0 ⇒ v0.3.1
  [872c559c] ↑ NNlib v0.4.2 ⇒ v0.4.3
  [bac558e1] + OrderedCollections v1.0.2
  [276daf66] ↑ SpecialFunctions v0.7.0 ⇒ v0.7.2
  [90137ffa] ↑ StaticArrays v0.8.3 ⇒ v0.10.0
  [2913bbd2] ↑ StatsBase v0.25.0 ⇒ v0.26.0
  [a5390f91] ↑ ZipFile v0.7.0 ⇒ v0.8.0


This replicates the housing data example from the Knet.jl readme. Although we
could have reused more of Flux (see the mnist example), the library's
abstractions are very lightweight and don't force you into any particular
strategy.

In [2]:
cd(@__DIR__)

isfile("housing.data") ||
  download("https://raw.githubusercontent.com/MikeInnes/notebooks/master/housing.data",
           "housing.data")

rawdata = readdlm("housing.data")'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48985  100 48985    0     0   349k      0 --:--:-- --:--:-- --:--:--  349k


14×505 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
   0.02731    0.02729    0.03237  …    0.06076    0.10959    0.04741
   0.0        0.0        0.0           0.0        0.0        0.0    
   7.07       7.07       2.18         11.93      11.93      11.93   
   0.0        0.0        0.0           0.0        0.0        0.0    
   0.469      0.469      0.458         0.573      0.573      0.573  
   6.421      7.185      6.998    …    6.976      6.794      6.03   
  78.9       61.1       45.8          91.0       89.3       80.8    
   4.9671     4.9671     6.0622        2.1675     2.3889     2.505  
   2.0        2.0        3.0           1.0        1.0        1.0    
 242.0      242.0      222.0         273.0      273.0      273.0    
  17.8       17.8       18.7      …   21.0       21.0       21.0    
 396.9      392.83     394.63        396.9      393.45     396.9    
   9.14       4.03       2.94          5.64       6.48       7.88   
  21.6       34.7       33.4          23.9     

The last feature is our target -- the price of the house.

In [3]:
x = rawdata[1:13,:]
y = rawdata[14:14,:]

1×505 Array{Float64,2}:
 21.6  34.7  33.4  36.2  28.7  22.9  …  16.8  22.4  20.6  23.9  22.0  11.9

Normalise the data

In [4]:
x = (x .- mean(x, dims = 2)) ./ std(x, dims = 2)

13×505 Array{Float64,2}:
 -0.417416  -0.417418  -0.416828  …  -0.41353   -0.407858  -0.415081
 -0.486234  -0.486234  -0.486234     -0.486234  -0.486234  -0.486234
 -0.595732  -0.595732  -1.30899       0.11315    0.11315    0.11315 
 -0.272618  -0.272618  -0.272618     -0.272618  -0.272618  -0.272618
 -0.739098  -0.739098  -0.833934      0.15753    0.15753    0.15753 
  0.194741   1.28121    1.01528   …   0.983996   0.725177  -0.361293
  0.366208  -0.265527  -0.808535      0.795646   0.735312   0.433641
  0.556346   0.556346   1.0759       -0.771891  -0.66685   -0.611768
 -0.868939  -0.868939  -0.754097     -0.983782  -0.983782  -0.983782
 -0.987128  -0.987128  -1.10573      -0.803294  -0.803294  -0.803294
 -0.306024  -0.306024   0.110158  …   1.17373    1.17373    1.17373 
  0.441136   0.396591   0.416291      0.441136   0.403377   0.441136
 -0.494157  -1.20985   -1.36251      -0.984357  -0.866709  -0.670629

The model

In [5]:
W = param(randn(1,13)/10)
b = param([0.])

Tracked 1-element Array{Float64,1}:
 0.0

using CuArrays
W, b, x, y = cu.((W, b, x, y))

In [6]:
predict(x) = W*x .+ b
meansquarederror(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)
loss(x, y) = meansquarederror(predict(x), y)

η = 0.1
θ = Params([W, b])

for i = 1:10
  g = gradient(() -> loss(x, y), θ)
  for x in θ
    update!(x, -g[x]*η)
  end
  @show loss(x, y)
end

predict(x[:,1]) / y[1]

loss(x, y) = 367.2818703136513 (tracked)
loss(x, y) = 242.00358836680243 (tracked)
loss(x, y) = 163.10915159227505 (tracked)
loss(x, y) = 112.83224080945294 (tracked)
loss(x, y) = 80.72250651347507 (tracked)
loss(x, y) = 60.18435464121343 (tracked)
loss(x, y) = 47.02585643472023 (tracked)
loss(x, y) = 38.578445205395504 (tracked)
loss(x, y) = 33.141510547752965 (tracked)
loss(x, y) = 29.630301742513243 (tracked)


Tracked 1-element Array{Float64,1}:
 1.0565988408749545